In [70]:
from IPython.display import display, HTML

display(HTML('''
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Code"></form>
'''))

import pandas as pd
import folium
import searoute as sr
from folium.plugins import FeatureGroupSubGroup
import branca.colormap as cm
from IPython.display import display, Markdown
import math
from datetime import datetime

# Get today's date
today_date = datetime.today().strftime("%B %d, %Y")  # Example: "February 10, 2024"

# Display the title using Markdown above the map
display(Markdown(f"""
<div style="text-align: center;">
    <h1>Baltic Handy Index Routes</h1>
    <h3>{today_date}</h3>
</div>
"""))

# Read the Excel file
df = pd.read_excel('Handy Baltic Rates.xlsx', engine='openpyxl')

# Ensure correct column headers are set
df.columns = df.iloc[1]  # Set second row as column headers
df = df[2:].reset_index(drop=True)  # Remove first two rows and reset index

# Drop rows where all elements are NaN
df = df.dropna(how='all')

# Drop columns where all values are NaN
df = df.dropna(axis=1, how='all')

# Ensure 'Value' column exists
if 'Value' not in df.columns:
    raise KeyError(f"The column 'Value' was not found. Available columns: {df.columns.tolist()}")

# Fix 'Value' column: Remove extra text, replace NaNs with 0, and convert to integer
df['Value'] = df['Value'].astype(str).str.strip().str.replace(' $/day', '', regex=False)
df['Value'] = df['Value'].replace(['nan', ''], 0).fillna(0).astype(int)

# **Fix Latitude & Longitude Columns**
lat_lon_columns = ['Origin Lat', 'Origin Long', 'Destination Lat', 'Destination Long']

for col in lat_lon_columns:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].str.replace(r'[^0-9.-]', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df[lat_lon_columns] = df[lat_lon_columns].fillna(0)

# Rename columns for easier reference
df = df.rename(columns={
    'Route': 'route',
    'Description': 'description',
    'Size (MT)': 'size_mt',  # Keeping as string
    'Value': 'value',
    'Change': 'change',
    'Origin Lat': 'origin_lat',
    'Origin Long': 'origin_long',
    'Destination Lat': 'destination_lat',
    'Destination Long': 'destination_long'
})

# Ensure 'size_mt' remains a string
df['size_mt'] = df['size_mt'].astype(str)

# Convert 'value' to integer
df['value'] = df['value'].astype(int)

# **Recalculate Average Coordinates for Map Centering**
average_lat = df[['origin_lat', 'destination_lat']].mean().mean()
average_long = df[['origin_long', 'destination_long']].mean().mean()

# Create a Folium map centered at the average location of all routes
m = folium.Map(location=[average_lat, average_long], zoom_start=3)

# Feature Group for route layers
routes_layer = folium.FeatureGroup(name="Sea Routes").add_to(m)

# **Create a color gradient from red to green based on 'value'**
min_value = df['value'].min()
max_value = df['value'].max()
colormap = cm.linear.RdYlGn_09.scale(min_value, max_value)

# Iterate through voyages and plot routes
for voyage in df.to_dict(orient='records'):
    origin = (voyage['origin_long'], voyage['origin_lat'])
    destination = (voyage['destination_long'], voyage['destination_lat'])

    # Assign a color based on the "value"
    color = colormap(voyage['value'])

    # Use searoute to get actual sea path
    try:
        route_response = sr.searoute(origin, destination)
        route_coords = route_response.get("geometry", {}).get("coordinates", [])

        # Ensure valid route data
        if not route_coords or not isinstance(route_coords, list):
            continue

        # Convert (long, lat) → (lat, long) for Folium
        route_coords = [(float(lat), float(lon)) for lon, lat in route_coords]

    except Exception:
        continue

    # Create route layer
    route_group = FeatureGroupSubGroup(routes_layer, name=voyage['route'])

    # Add polyline to map
    folium.PolyLine(
        locations=route_coords,
        color=color,
        weight=3,
        opacity=0.7,
        popup=folium.Popup(f"Route: {voyage['route']}<br>Value: ${voyage['value']}", max_width=300),
        tooltip=f"Value: ${voyage['value']}"
    ).add_to(route_group)

    # **Add One Scalable Arrow at 75% of the Route**
    arrow_idx = int(len(route_coords) * 0.75)  # Get the index 75% along the route
    if arrow_idx < len(route_coords) - 1:
        arrow_tip = route_coords[arrow_idx]
        next_point = route_coords[arrow_idx + 1]

        # Calculate angle for arrow direction
        angle = math.degrees(math.atan2(next_point[1] - arrow_tip[1], next_point[0] - arrow_tip[0]))

        # **Use a marker cluster to adjust size dynamically**
        folium.Marker(
            location=arrow_tip,
            icon=folium.DivIcon(html=f"""
                <svg width="20" height="20" viewBox="-10 -10 20 20">
                    <line x1="0" y1="8" x2="0" y2="-8" stroke="black" stroke-width="3" transform="rotate({angle}, 0, 0)"/>
                    <polygon points="-5,-2 5,-2 0,-10" fill="black" transform="rotate({angle}, 0, 0)"/>
                </svg>
            """),
        ).add_to(route_group)

    # **Label the route with the value (at midpoint)**
    mid_idx = len(route_coords) // 2
    mid_point = route_coords[mid_idx]

    folium.Marker(
        location=mid_point,
        icon=folium.DivIcon(html=f"<div style='font-size: 12px; color: black; font-weight: bold;'>{voyage['value']}</div>")
    ).add_to(route_group)

    route_group.add_to(m)

# **Add Color Legend to the Map**
colormap.caption = "Voyage Value (Price)"
m.add_child(colormap)

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Display the map in Jupyter Notebook
display(m)



<div style="text-align: center;">
    <h1>Baltic Handy Index Routes</h1>
    <h3>February 06, 2025</h3>
</div>
